## EDA+关联规则——就业城市与薪资的关系

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyecharts.charts import Bar
from pyecharts.charts import Pie
from pyecharts.charts import Geo
from pyecharts.charts import HeatMap
from pyecharts.faker import Faker
from pyecharts import options as opts
from pyecharts.charts import Scatter3D
from pyecharts.charts import Line
import random
import math

In [12]:
data = pd.read_csv('../../../data/info_o.csv')

在1.1节中，我们已经看到了这些大数据企业的地理分布，呈现出围绕着经济中心逐渐下降，从沿海到内地逐渐下降等趋势。

从大数据岗位的招聘信息数量上来看，这些企业遍布全国208座城市，且不同城市对大数据人才的需求程度差异悬殊，需求量较大的城市能达到几千则招聘信息，如北京高达4797；而也有许多需求量小的城市，可能在数据集中只出现过1则。

In [13]:
print("大数据岗位招聘信息量Top30:")
data['w_place'].value_counts().head(30)

大数据岗位招聘信息量Top30:


北京     4797
深圳     2016
杭州     1455
上海     1185
广州     1037
成都      977
南京      707
武汉      691
郑州      532
西安      502
济南      391
合肥      364
长沙      352
苏州      316
重庆      305
福州      290
厦门      276
青岛      256
贵阳      250
沈阳      233
天津      195
哈尔滨     189
石家庄     175
长春      164
大连      157
太原      129
无锡      113
南昌      107
昆明      105
佛山       79
Name: w_place, dtype: int64

对比大数据岗位招聘信息发布量的前30座城市，再次印证了刚才的观点，即各城市间对大数据人才的需求程度差异非常大。北京遥遥领先，深圳紧随其后，其次是杭州、上海、广州，均在1000以上。其他二十几座城市的大数据人才招聘信息数分布在100-1000之间。

In [14]:
firm = data[['c_name','c_nature','c_scale','w_place']].drop_duplicates(subset=['c_name'],keep='first')

In [15]:
df = firm['w_place'].value_counts().head(10)
x = list(df.index.values)
y = [int(x) for x in df.values]
pie = Pie()
pie.add("", [list(z) for z in zip(x, y)])
pie.render_notebook()

再来看一下各城市的大数据企业分布。从占比上来看，拥有大数据企业数量前十的城市，皆是一线城市和新一线城市，这也正照应了上述大数据岗位招聘信息数排名情况：大数据企业越多，对大数据人才的需求越大。

其中，北京的大数据企业最多，达到2000家，其次是深圳、杭州、广州。这是符合事实的，随着大数据国家战略的加速落地，大数据体量呈现爆发式增长，这些一线城市、新一线城市在大数据行业的发展上处于领先态势，急需招揽大量的大数据人才。正如我们所知道的，许多知名大数据企业正是坐落在上述城市。

In [16]:
import json
with open('city_to_province.json',encoding='utf-8') as f:
    d = json.load(f)

In [17]:
data=data.drop(data[data.w_place=='其他'].index)

In [18]:
pro=[]
for i in range(0, len(data)):
    pro.append(d[data.iloc[i]['w_place']])
    
#增加省份列
data.loc[:,'w_pro']=pro

In [19]:
data[['w_place','w_pro']]

,w_place,w_pro
6,阿拉善盟,内蒙古自治区
7,安庆,安徽省
8,包头,内蒙古自治区
9,包头,内蒙古自治区
10,包头,内蒙古自治区
...,...,...
19853,北京,北京市
19854,北京,北京市
19855,北京,北京市
19856,南京,江苏省


In [20]:
city_gp =data.pivot_table('s_average',index='w_place')
x = list(city_gp.index.values)
y = [int(x) for x in city_gp.values]

In [22]:
m = Geo()
m.add_schema(maptype="china",
             itemstyle_opts=opts.ItemStyleOpts(color="#404a59", border_color="#fff11"),
            )
m.add_coordinate_json('city_location.json')
m.add("", [list(z) for z in zip(x, y)],symbol_size=8)
m.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
m.set_global_opts(
    visualmap_opts=opts.VisualMapOpts(min_=6,max_=15)
)
m.render_notebook()

从地理分布上来看，各城市对于大数据岗位开出的平均薪资，与各城市大数据企业的地理分布趋势相似。沿海地区城市的薪资相对更高，内陆地区相对较低。

In [27]:
pro_gp = data.pivot_table('s_average',index='w_pro')
pro_gp = pro_gp[pro_gp.index!=""]
pro_gp = pro_gp.sort_values(by='s_average',ascending=False)

x = list(pro_gp.index.values)
y = [int(x) for x in pro_gp.values]
bar = Bar()
bar.add_xaxis(x)
bar.add_yaxis("",y,color= "#2c85ff")
bar.set_global_opts(xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-65)))
bar.render_notebook()

In [26]:
pro_gp[pro_gp.index!=""]

,s_average
w_pro,
北京市,21.777152
上海市,21.531646
澳门,17.500000
浙江省,16.690176
广东省,16.343220
福建省,13.036184
江苏省,12.172439
四川省,12.057385
西藏自治区,12.000000


柱状图反映了各省的平均薪资排名。

·北京、上海的大数据招聘岗位给出的平均薪资最高，为21k左右。

·东北三省的平均薪资为7-8k，在各省份中处于较低水平。

·从沿海省份到内陆，薪资水平逐渐降低，并呈现阶梯状分布。

In [28]:
gp = data.pivot_table('s_average',index='c_nature',columns='w_pro',aggfunc='mean',margins=True,fill_value=0)

In [12]:
def draw_heat(gp,v_max):
    value = [[i,j,round(gp.values[i][j])] for i in range(gp.index.shape[0])for j in range(gp.columns.shape[0])]
    c = HeatMap()
    c.add_xaxis(list(gp.index))
    c.add_yaxis(
            "",
            list(gp.columns),
            value,
            label_opts=opts.LabelOpts(is_show=True, position="inside"),
        )
    c.set_global_opts(
            title_opts=opts.TitleOpts(title=""),
            visualmap_opts=opts.VisualMapOpts(min_=0, max_=v_max),
            xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-45))
        )
    return c

In [13]:
c = draw_heat(gp,gp.values.max())
c.render_notebook()

上图所示为各省份不同性质的大数据企业，对大数据岗位给出的平均薪资(单位k/月)。从图中可以看出：

·1.对于一些大数据发展较成熟的省份，如北京、上海、广东等，能够对大数据岗位开出的薪资明显较高，平均在22k，且基本覆盖各种性质的企业。

·2.像江浙、福建、山东等沿海省份，依托沿海的优势，虽然薪资水平略低于一线城市，10-17k之间都有分布，但在各种企业间也基本都有所涉及。

·3.广东、福建的港澳台公司贡献了当地较高的大数据岗位薪资，达到18k-20k，这说明当地与港澳台合作的大数据企业发展迅速，且对大数据人才的需求急切。

·4.内陆省份一般都会有大数据发展突出的某种类型的企业，例如河南省的外商独资、甘肃省与陕西省的代表处、新疆维吾尔自治区的上市公司、西藏自治区的国企等，能够开出18k左右，甚至高达30k的月薪，可以说是当地大数据企业中的“领头羊”。

##### 关联规则挖掘

In [14]:
data_h = pd.read_csv('../../../data/info_h.csv')
c_s = data_h[[ 'w_place','c_nature', 's_average']]

In [15]:
from mlxtend.preprocessing import TransactionEncoder
def deal(data):
    return data.to_list()
def encode(df):
    df_arr = df.apply(deal,axis=1).tolist()
    te = TransactionEncoder()
    df_tf = te.fit_transform(df_arr)
    df = pd.DataFrame(df_tf,columns=te.columns_)
    return df

In [16]:
c_s = encode(c_s)

In [17]:
from mlxtend.frequent_patterns import apriori
frequent_items = apriori(c_s, min_support=0.05, use_colnames=True, max_len=4).sort_values(by='support', ascending=False)	
frequent_items.head(10)

,support,itemsets
10,0.676705,(民营)
3,0.357085,(a4)
2,0.293182,(a3)
18,0.284722,"(a4, 民营)"
6,0.241565,(北京)
1,0.231796,(a2)
21,0.184711,"(北京, 民营)"
16,0.181136,"(民营, a3)"
17,0.160892,"(a4, 北京)"
14,0.144425,"(a2, 民营)"


In [18]:
# 评价方法
import math
def metrics(r,f):
    ans = []
    for i in range(r.shape[0]):
        item = r.iloc[i]
        ans.append(f(item))
    return ans
def allconf(item):
    return item.support/max(item['antecedent support'],item['consequent support'])
def cosine(item):
    return item.support/math.sqrt(item['antecedent support']*item['consequent support'])
def Jaccard(item):
    return item.support/(item['antecedent support']+item['consequent support']-item.support)
def maxconf(item):
    return max(item.support/item['antecedent support'],item.support/item['consequent support'])
def Kulczynski(item):
    return 0.5*(item.support/item['antecedent support']+item.support/item['consequent support'])

In [19]:
from mlxtend.frequent_patterns import association_rules
def get_rules(frequent_items):
    rules =  association_rules(frequent_items, metric='lift')
    rules = rules.sort_values(by=['lift'], ascending=False).reset_index(drop=True)
    rules = rules.drop(['leverage','conviction'],axis = 1)
    rules['cosine'] = metrics(rules,cosine)
    rules['Jaccard'] = metrics(rules,Jaccard)
    rules['Allconf'] = metrics(rules,allconf)
    rules['Maxconf'] = metrics(rules,maxconf)
    rules['Kulczynski'] = metrics(rules,Kulczynski)
    return rules

In [20]:
rules = get_rules(frequent_items)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,cosine,Jaccard,Allconf,Maxconf,Kulczynski
0,(a4),"(北京, 民营)",0.357085,0.184711,0.132692,0.371598,2.011775,0.516669,0.324348,0.371598,0.718375,0.544986
1,"(北京, 民营)",(a4),0.184711,0.357085,0.132692,0.718375,2.011775,0.516669,0.324348,0.371598,0.718375,0.544986
2,"(a4, 民营)",(北京),0.284722,0.241565,0.132692,0.466042,1.929259,0.505962,0.337129,0.466042,0.549302,0.507672
3,(北京),"(a4, 民营)",0.241565,0.284722,0.132692,0.549302,1.929259,0.505962,0.337129,0.466042,0.549302,0.507672
4,(北京),(a4),0.241565,0.357085,0.160892,0.666041,1.865216,0.547813,0.367537,0.450571,0.666041,0.558306
5,(a4),(北京),0.357085,0.241565,0.160892,0.450571,1.865216,0.547813,0.367537,0.450571,0.666041,0.558306
6,(a4),(深圳),0.357085,0.101521,0.051063,0.142998,1.408560,0.268188,0.125293,0.142998,0.502976,0.322987
7,(深圳),(a4),0.101521,0.357085,0.051063,0.502976,1.408560,0.268188,0.125293,0.142998,0.502976,0.322987
8,(上海),(民营),0.059674,0.676705,0.055695,0.933333,1.379233,0.277159,0.081823,0.082304,0.933333,0.507819
9,(民营),(上海),0.676705,0.059674,0.055695,0.082304,1.379233,0.277159,0.081823,0.082304,0.933333,0.507819


导出的关联规则说明：

·1.北京的民营企业通常对大数据行业岗位开出平均17k左右的薪资；

·2.北京和深圳的大数据企业对招聘岗位一般能够给出17k左右的薪资；

·3.在大数据人才需求量较大、大数据企业聚集的这几座城市中，即北京、深圳、上海、杭州，民营企业都占了较大比例，所开出的薪资范围广泛，从6k、7-10k、11-16k、17k都有涉及，可见民营企业在当地的大数据行业群体内占据了非常重要的位置；


In [21]:
def draw_scatter(rules):
    # 配置 config
    config_xAxis3D = "support"
    config_yAxis3D = "confidence"
    config_zAxis3D = "lift"
    config_color = "support"
    config_symbolSize = "lift"
    # 构造数据
    data = [
        [
            rules.loc[i][config_xAxis3D],
            rules.loc[i][config_yAxis3D],
            rules.loc[i][config_zAxis3D],
            rules.loc[i][config_color],
            rules.loc[i][config_symbolSize],
            i,
        ]
        for i in range(rules.shape[0])
    ]

    s3= Scatter3D(init_opts=opts.InitOpts(width="700px", height="320px"))  # bg_color="black"
    s3.add(
        series_name="",
        data=data,
        xaxis3d_opts=opts.Axis3DOpts(
            name=config_xAxis3D,
            type_="value"
        ),
        yaxis3d_opts=opts.Axis3DOpts(
            name=config_yAxis3D,
            type_="value"
        ),
        zaxis3d_opts=opts.Axis3DOpts(
            name=config_zAxis3D,
            type_="value"
        ),
        grid3d_opts=opts.Grid3DOpts(width=100, height=100, depth=100),
    )
    s3.set_global_opts(
        visualmap_opts=[
            opts.VisualMapOpts(
                type_="color",
                is_calculable=True,
                dimension=3,
                pos_top="10",
                max_= 0.5,
                range_color=[
                        "#1710c0",
                        "#0b9df0",
                        "#00fea8",
                        "#00ff0d",
                        "#f5f811",
                        "#f09a09",
                        "#fe0300",
                ],
            )
        ]
    )
    return s3

 **规则评价**
  
下图给出了导出关联规则的置信度、支持度以及Lift的散点图。

In [22]:
s = draw_scatter(rules)
s.render_notebook()